In [ ]:
from music21 import converter, note, stream, chord
from collections import defaultdict
import random

def tokenize_note(note_obj):
    if isinstance(note_obj, note.Note):
        pitch = note_obj.pitch.nameWithOctave
        duration = note_obj.duration.quarterLength
        return f"{pitch}_{duration}"
    elif isinstance(note_obj, note.Rest):
        return "Rest"
    elif isinstance(note_obj, chord.Chord):
        tokens = []
        for pitch in note_obj.pitches:
            pitch_name = pitch.nameWithOctave
            duration = note_obj.duration.quarterLength
            tokens.append(f"{pitch_name}_{duration}")
        if tokens:
            return "/".join(tokens)  # Join chord tokens with "/"
        else:
            return "Chord"  # Return a placeholder token for empty chords
    else:
        return ""  # Return an empty string for unrecognized note objects

def tokenize_notes(notes):
    tokens = []
    for note_obj in notes:
        token = tokenize_note(note_obj)
        if token:
            tokens.append(token)
    return tokens

import os
import glob


def generate_markov_model(folder_path, order):
    # Create an empty list to store all the notes from the MIDI files
    all_notes = []

    # Iterate over MIDI files in the folder
    for file_path in glob.glob(os.path.join(folder_path, "*.mid")):
        # Load MIDI file
        midi_stream = converter.parse(file_path)
        # Extract notes from MIDI file
        notes = midi_stream.flat.notes
        # Tokenize notes
        note_tokens = tokenize_notes(notes)
        # Append note tokens to the list
        all_notes.extend(note_tokens)

    # Create transition table
    transition_table = defaultdict(lambda: defaultdict(int))

    # Iterate over note tokens to build transition table
    for i in range(len(all_notes) - order):
        current_state = tuple(all_notes[i+j] for j in range(order))
        next_state = all_notes[i+order]
        transition_table[current_state][next_state] += 1

    # Normalize transition probabilities
    for current_state in transition_table:
        total_count = sum(transition_table[current_state].values())
        for next_state in transition_table[current_state]:
            transition_table[current_state][next_state] /= total_count

    return transition_table


def generate_new_music(transition_table, order, length, seed_notes):
    # Set the initial state as the seed notes
    current_state = tuple(tokenize_note(note_obj) for note_obj in seed_notes)

    # Create a stream to store the generated notes
    generated_stream = stream.Stream()

    # Append the seed notes to the generated stream
    for note_obj in seed_notes:
        generated_stream.append(note_obj)

    # Generate new notes based on transition probabilities
    for _ in range(length - len(seed_notes)):
        if current_state not in transition_table:
            # If current state is not in the transition table, select a random state
            current_state = random.choice(list(transition_table.keys()))

        next_note_str = random.choice(list(transition_table[current_state].keys()))


        if next_note_str == "Rest":
            next_note = note.Rest()
        elif next_note_str == "Chord":  # Handle empty chords
            next_note = chord.Chord()
        elif next_note_str and "/" in next_note_str:  # Handle chord duration
            chord_tokens = next_note_str.split("/")
            chord_notes = []
            duration_parts = chord_tokens[0].split("_")
            duration_value = float(duration_parts[-1]) if len(duration_parts) > 1 else 1.0
            for chord_token in chord_tokens:
                if chord_token:
                    pitch = chord_token.split("_")[0]
                    try:
                        chord_note = note.Note(pitch)
                        chord_note.duration.quarterLength = duration_value
                        chord_notes.append(chord_note)
                    except Exception:
                        continue
            next_note = chord.Chord(chord_notes)
        elif next_note_str:  # Handle regular note
            pitch, duration_str = next_note_str.split("_", 1)  # Split using the first underscore only
            duration_parts = duration_str.split("/")
            duration_value = float(duration_parts[0]) / float(duration_parts[1]) if len(duration_parts) == 2 else float(duration_parts[0])
            try:
                next_note = note.Note(pitch)
                next_note.duration.quarterLength = duration_value
            except Exception:
                # If pitch is invalid, select a random state
                current_state = random.choice(list(transition_table.keys()))
                continue
        else:
            # If next_note_str is an empty string, select a random state
            current_state = random.choice(list(transition_table.keys()))
            continue

        generated_stream.append(next_note)

        # Update current state
        current_state = current_state[1:] + (next_note_str,)

    return generated_stream


def print_transition_table(transition_table):
    for current_state in transition_table:
        print(f"Current state: {current_state}")
        for next_state, probability in transition_table[current_state].items():
            print(f"Next state: {next_state} Probability: {probability}")
        print("-" * 20)


def extract_seed_from_midi(seed_file_path):
    # Load MIDI file
    midi_stream = converter.parse(seed_file_path)

    # Extract the first "Choose #" of notes
    seed_notes = []
    for note_obj in midi_stream.flat.notes:
        if len(seed_notes) >= 16:
            break
        seed_notes.append(note_obj)

    return seed_notes

# Set the folder path containing MIDI files INPUT #1
folder_path = "/content/Jazz"

# Set the order of the Markov model (number of previous notes to consider)
#I used 3 for Jazz, and "" for Classical
order = 3

# Set the length of the generated music (in notes).  used 200 for the output
generated_length = 200

# Generate the Markov model
transition_table = generate_markov_model(folder_path, order)

# Set the path to the MIDI file containing the seed notes INPUT #2
seed_file_path = "/content/Jazz Seed 1.mid"

# Extract the seed notes from the MIDI file
seed_notes = extract_seed_from_midi(seed_file_path)

# Print the transition table
print_transition_table(transition_table)

# Generate new music based on the transition table
generated_music = generate_new_music(transition_table, order, generated_length, seed_notes)

# Save the generated music as a MIDI file
output_file_path = "output.mid"
generated_music.write("midi", fp=output_file_path)
print(f"Generated music saved as '{output_file_path}'.")
